In [1]:
import sys
sys.path


['/home/mnt/nzh/nzhanglab/project/shuang/scATAC/comparison_methods/scglue',
 '/usr/local/python-3.8.5/lib/python38.zip',
 '/usr/local/python-3.8.5/lib/python3.8',
 '/usr/local/python-3.8.5/lib/python3.8/lib-dynload',
 '',
 '/home/stat/shuang91/gpuenv385/lib/python3.8/site-packages',
 '/home/stat/shuang91/gpuenv385/lib/python3.8/site-packages/stereoscope-0.3.1-py3.8.egg']

In [2]:
sys.path.append('/home/mnt/nzh/nzhanglab/project/shuang/miniconda3/envs/scglue2/lib/python3.8/site-packages')



In [3]:
import anndata
import networkx as nx
import scanpy as sc
import scglue
from matplotlib import rcParams

In [4]:
from datetime import datetime
time_now = datetime.now()
current_time = time_now.strftime("%H:%M:%S")

print("The current date and time is", current_time)

The current date and time is 18:29:27


In [5]:
rna = anndata.read_h5ad("retina/retina_RNA.2000.h5ad")
rna


AnnData object with n_obs × n_vars = 2500 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'celltype', 'domain'
    var: 'features'

In [6]:
atac = anndata.read_h5ad("retina/retina_ATAC.2000.h5ad")
atac

AnnData object with n_obs × n_vars = 2500 × 325401
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'celltype', 'domain'
    var: 'features'

In [7]:
rna.X, rna.X.data


(<2500x36601 sparse matrix of type '<class 'numpy.float64'>'
 	with 4370242 stored elements in Compressed Sparse Row format>,
 array([1., 1., 2., ..., 1., 1., 1.]))

In [8]:
rna.layers["counts"] = rna.X.copy()


In [9]:
sc.pp.highly_variable_genes(rna, n_top_genes=2000, flavor="seurat_v3")


In [10]:
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
sc.pp.scale(rna)
sc.tl.pca(rna, n_comps=100, svd_solver="auto")

In [12]:
atac.X, atac.X.data


(<2500x325401 sparse matrix of type '<class 'numpy.float64'>'
 	with 22494323 stored elements in Compressed Sparse Row format>,
 array([2., 2., 1., ..., 1., 2., 2.]))

In [13]:
scglue.data.lsi(atac, n_components=100, n_iter=15)


/home/mnt/nzh/nzhanglab/project/shuang/miniconda3/envs/scglue2/lib/python3.8/site-packages/scglue/num.py:289: RuntimeWarning: divide by zero encountered in true_divide
  idf = X.shape[0] / X.sum(axis=0)
/home/stat/shuang91/gpuenv385/lib/python3.8/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


In [14]:
sc.pp.neighbors(atac, use_rep="X_lsi", metric="cosine")
sc.tl.umap(atac)

In [16]:
rna.var.head()


,features,highly_variable,highly_variable_rank,means,variances,variances_norm,mean,std
MIR1302-2HG,MIR1302-2HG,False,NaN,0.0024,0.002395,0.876998,0.001088,0.024788
FAM138A,FAM138A,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000
OR4F5,OR4F5,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000
AL627309.1,AL627309.1,False,NaN,0.0056,0.005571,0.825701,0.003545,0.054385
AL627309.3,AL627309.3,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000


In [17]:
rna.var_names.name = 'genes'

In [18]:
rna.var_names

Index(['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3',
       'AL627309.2', 'AL627309.5', 'AL627309.4', 'AP006222.2', 'AL732372.1',
       ...
       'AC133551.1', 'AC136612.1', 'AC136616.1', 'AC136616.3', 'AC136616.2',
       'AC141272.1', 'AC023491.2', 'AC007325.1', 'AC007325.4', 'AC007325.2'],
      dtype='object', name='genes', length=36601)

In [19]:
scglue.data.get_gene_annotation(
    rna, gtf="gencode.v40.chr_patch_hapl_scaff.annotation.gtf.gz",
    gtf_by="gene_name"
)
rna.var.loc[:, ["chrom", "chromStart", "chromEnd"]].head()

,chrom,chromStart,chromEnd
genes,,,
MIR1302-2HG,chr1,29553.0,31109.0
FAM138A,chr1,34553.0,36081.0
OR4F5,chr1,65418.0,71585.0
AL627309.1,NaN,NaN,NaN
AL627309.3,NaN,NaN,NaN


In [20]:
rna.var

,features,highly_variable,highly_variable_rank,means,variances,variances_norm,mean,std,chrom,chromStart,...,thickEnd,itemRgb,blockCount,blockSizes,blockStarts,gene_id,gene_type,hgnc_id,havana_gene,tag
genes,,,,,,,,,,,,,,,,,,,,,
MIR1302-2HG,MIR1302-2HG,False,NaN,0.0024,0.002395,0.876998,0.001088,0.024788,chr1,29553.0,...,.,.,.,.,.,ENSG00000243485.5,lncRNA,HGNC:52482,OTTHUMG00000000959.2,ncRNA_host
FAM138A,FAM138A,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,chr1,34553.0,...,.,.,.,.,.,ENSG00000237613.2,lncRNA,HGNC:32334,OTTHUMG00000000960.1,NaN
OR4F5,OR4F5,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,chr1,65418.0,...,.,.,.,.,.,ENSG00000186092.7,protein_coding,HGNC:14825,OTTHUMG00000001094.4,NaN
AL627309.1,AL627309.1,False,NaN,0.0056,0.005571,0.825701,0.003545,0.054385,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AL627309.3,AL627309.3,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC141272.1,AC141272.1,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AC023491.2,AC023491.2,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AC007325.1,AC007325.1,False,NaN,0.0004,0.000400,0.999718,0.000098,0.004923,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df = rna.var
#print(df.dropna(subset=['chrom'])["features"])

In [22]:
rna2 = rna
rna2 = rna2[:,df.dropna(subset=['chrom'])["features"]]

In [23]:
rna2.var.loc[:, ["chrom", "chromStart", "chromEnd"]]

,chrom,chromStart,chromEnd
genes,,,
MIR1302-2HG,chr1,29553.0,31109.0
FAM138A,chr1,34553.0,36081.0
OR4F5,chr1,65418.0,71585.0
OR4F29,chr1,450739.0,451678.0
OR4F16,chr1,685715.0,686654.0
...,...,...,...
MT-ND4,chrM,10759.0,12137.0
MT-ND5,chrM,12336.0,14148.0
MT-ND6,chrM,14148.0,14673.0


In [24]:
rna2.var_names

Index(['MIR1302-2HG', 'FAM138A', 'OR4F5', 'OR4F29', 'OR4F16', 'LINC01409',
       'FAM87B', 'LINC01128', 'LINC00115', 'FAM41C',
       ...
       'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4', 'MT-ND5',
       'MT-ND6', 'MT-CYB', 'MAFIP'],
      dtype='object', name='genes', length=23582)

In [25]:
#rna2.var.loc[:, ["chrom", "chromStart", "chromEnd"]].to_csv('e18mouse/rnavar2.csv',index=False)

In [26]:
atac.var_names[:5]


Index(['chr1-794327-794827', 'chr1-794856-795356', 'chr1-804383-804883',
       'chr1-817118-817618', 'chr1-819862-820362'],
      dtype='object')

In [27]:
split = atac.var_names.str.split(r"[:-]")
atac.var["chrom"] = split.map(lambda x: x[0])
atac.var["chromStart"] = split.map(lambda x: x[1])
atac.var["chromEnd"] = split.map(lambda x: x[2])
atac.var.head()

,features,chrom,chromStart,chromEnd
chr1-794327-794827,chr1-794327-794827,chr1,794327,794827
chr1-794856-795356,chr1-794856-795356,chr1,794856,795356
chr1-804383-804883,chr1-804383-804883,chr1,804383,804883
chr1-817118-817618,chr1-817118-817618,chr1,817118,817618
chr1-819862-820362,chr1-819862-820362,chr1,819862,820362


In [28]:
graph = scglue.genomics.rna_anchored_prior_graph(rna2, atac)
graph

/usr/local/python-3.8.5/lib/python3.8/subprocess.py:838: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/usr/local/python-3.8.5/lib/python3.8/subprocess.py:844: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
/usr/local/python-3.8.5/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)
/usr/local/python-3.8.5/lib/python3.8/subprocess.py:838: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/usr/local/python-3.8.5/lib/python3.8/subprocess.py:844: RuntimeWarning: line buffering (buffering=1) isn't supported in bin

  0%|          | 0/23582 [00:00<?, ?it/s]

In [29]:
graph.number_of_nodes(), graph.number_of_edges()


(348983, 874335)

In [30]:
# Graph node covers all omic features
all(graph.has_node(gene) for gene in rna2.var_names), \
all(graph.has_node(peak) for peak in atac.var_names)

(True, True)

In [31]:
# Edge attributes contain weights and signs
for _, e in zip(range(5), graph.edges):
    print(f"{e}: {graph.edges[e]}")

('LINC01409', 'chr1-794327-794827', 0): {'dist': 0, 'weight': 1.0, 'sign': 1, 'type': 'fwd'}
('LINC01409', 'chr1-794856-795356', 0): {'dist': 0, 'weight': 1.0, 'sign': 1, 'type': 'fwd'}
('LINC01409', 'chr1-804383-804883', 0): {'dist': 0, 'weight': 1.0, 'sign': 1, 'type': 'fwd'}
('LINC01409', 'LINC01409', 0): {'weight': 1.0, 'sign': 1, 'type': 'loop'}
('chr1-794327-794827', 'LINC01409', 0): {'dist': 0, 'weight': 1.0, 'sign': 1, 'type': 'rev'}


In [32]:
# Each node has a self-loop
all(graph.has_edge(gene, gene) for gene in rna2.var_names), \
all(graph.has_edge(peak, peak) for peak in atac.var_names)


(True, True)

In [33]:
# Graph is symmetric
all(graph.has_edge(j, i) for i, j, _ in graph.edges)


True

In [34]:
atac.var.head()


,features,chrom,chromStart,chromEnd,highly_variable
chr1-794327-794827,chr1-794327-794827,chr1,794327,794827,False
chr1-794856-795356,chr1-794856-795356,chr1,794856,795356,False
chr1-804383-804883,chr1-804383-804883,chr1,804383,804883,False
chr1-817118-817618,chr1-817118-817618,chr1,817118,817618,False
chr1-819862-820362,chr1-819862-820362,chr1,819862,820362,False


In [35]:
rna = rna2

In [36]:
rna.var

,features,highly_variable,highly_variable_rank,means,variances,variances_norm,mean,std,chrom,chromStart,...,thickEnd,itemRgb,blockCount,blockSizes,blockStarts,gene_id,gene_type,hgnc_id,havana_gene,tag
genes,,,,,,,,,,,,,,,,,,,,,
MIR1302-2HG,MIR1302-2HG,False,NaN,0.0024,0.002395,0.876998,0.001088,0.024788,chr1,29553.0,...,.,.,.,.,.,ENSG00000243485.5,lncRNA,HGNC:52482,OTTHUMG00000000959.2,ncRNA_host
FAM138A,FAM138A,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,chr1,34553.0,...,.,.,.,.,.,ENSG00000237613.2,lncRNA,HGNC:32334,OTTHUMG00000000960.1,NaN
OR4F5,OR4F5,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,chr1,65418.0,...,.,.,.,.,.,ENSG00000186092.7,protein_coding,HGNC:14825,OTTHUMG00000001094.4,NaN
OR4F29,OR4F29,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,chr1,450739.0,...,.,.,.,.,.,ENSG00000284733.2,protein_coding,HGNC:31275,OTTHUMG00000002860.3,NaN
OR4F16,OR4F16,False,NaN,0.0000,0.000000,0.000000,0.000000,1.000000,chr1,685715.0,...,.,.,.,.,.,ENSG00000284662.2,protein_coding,HGNC:15079,OTTHUMG00000002581.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ND4,MT-ND4,False,NaN,0.1260,0.150184,0.746812,0.065351,0.212767,chrM,10759.0,...,.,.,.,.,.,ENSG00000198886.2,protein_coding,HGNC:7459,NaN,NaN
MT-ND5,MT-ND5,False,NaN,0.0240,0.026635,0.843449,0.011898,0.094013,chrM,12336.0,...,.,.,.,.,.,ENSG00000198786.2,protein_coding,HGNC:7461,NaN,NaN
MT-ND6,MT-ND6,False,NaN,0.0032,0.003191,0.855367,0.001764,0.036288,chrM,14148.0,...,.,.,.,.,.,ENSG00000198695.2,protein_coding,HGNC:7462,NaN,NaN


In [37]:
rna2.raw = None

In [38]:
rna2.raw

In [39]:
#rna.write("e18mouse/rna_preprocessed.h5ad", compression="gzip")
rna2.write("retina/rna_preprocessed_2000.h5ad", compression="gzip")

atac.raw = None
atac.write("retina/atac_preprocessed_2000.h5ad", compression="gzip")
nx.write_graphml(graph, "retina/prior_2000.graphml.gz")

/home/stat/shuang91/gpuenv385/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'chrom' as categorical
/home/stat/shuang91/gpuenv385/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'score' as categorical
/home/stat/shuang91/gpuenv385/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. 

In [40]:
time_now = datetime.now()
current_time = time_now.strftime("%H:%M:%S")

print("The current date and time is", current_time)

The current date and time is 18:34:17


#rna.write("e18mouse/rna_preprocessed.h5ad", compression="gzip")
rna2.write("retina/rna_preprocessed.h5ad", compression="gzip")

atac.raw = None
atac.write("retina/atac_preprocessed.h5ad", compression="gzip")
nx.write_graphml(graph, "retina/prior.graphml.gz")